In [2]:
import pandas as pd

In [3]:
df_reviews = pd.read_parquet('user_reviews.parquet')
df_reviews.head(3)


,item_id,recommend,review,user_id,posted_year,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011,2
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011,2


In [4]:
df_items = pd.read_parquet('user_items.parquet')
df_items.head(3)

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,277
2,30,Day of Defeat,7,76561197970982479,277


In [5]:
df_games = pd.read_parquet('steams_game.parquet')
df_games.head(3)

,genres_clean,app_name,price,item_id,developer,release_year
88310,Action,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88311,Free to Play,Ironbound,0.00,643980,Secret Level SRL,2018
88312,Casual,Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017


In [6]:
df1 = pd.merge(df_games,df_reviews, on="item_id")
df1.sample(2)

,genres_clean,app_name,price,item_id,developer,release_year,recommend,review,user_id,posted_year,sentiment_analysis
42157,Action,Team Fortress 2,0.0,440,Valve,2007,True,got tardas,Cibsy,2013,1
23475,Action,PAYDAY 2: Hotline Miami,0.0,323500,OVERKILL - a Starbreeze Studio.,2014,True,"你以为光你能写评测吗,我也能写",xinyo,2014,1


In [7]:
df = pd.merge(df1,df_items, on="user_id")
df.sample(2)

,genres_clean,app_name,price,item_id_x,developer,release_year,recommend,review,user_id,posted_year,sentiment_analysis,item_id_y,item_name,playtime_forever,items_count
2398411,Action,Insurgency,9.99,222880,New World Interactive,2014,True,This Game Is AWESOME!!!! play with some mates ...,76561198102234446,2013,2,326960,Killing Floor - Toy Master,0,47
4805273,Action,Double Action: Boogaloo,0.00,317360,Double Action Factory,2014,True,มันได้ใจดีค้าาาา,ghostfps98,2015,1,94500,Back to the Future: Ep 2 - Get Tannen!,0,190


In [11]:
df.shape

(5589876, 15)


Verifico si se encuentran duplicados

In [9]:

df.duplicated(subset=df.columns).sum()

88830

In [10]:
df.drop_duplicates(keep= 'last', inplace=True)

Extraigo el dataset final, con todos los dataframes unidos

In [22]:
df.to_parquet("data_final.parquet")

------------------------------------------------------------------

Aca hago otro tipo de union

Realizo la union de todos los datasets. (Este es el que se usa en la mayoria de joins)

In [64]:
df_merge = pd.merge(df_items,df_reviews, how="left")

In [65]:
df_merge = pd.merge(df_games,df_merge, how="left")

In [69]:
df_merge.shape

(4268268, 14)

Elimino las filas que contiene valores nulos

In [62]:
df_merge = df_merge.dropna()

In [63]:
df_merge.shape

(31935, 14)

---------------------------------------------------------------------------------------------------

Armo un dataframe para el endpoint 4. La columna 'release_year' que contiene los años era de tipo string y para el endpoint 4 necesitaba que sea de tipo int

In [13]:
df_endpoint_4 = df[['developer', 'recommend', 'release_year', 'sentiment_analysis']]
#df_endpoint_4_5.to_parquet('endpoint_4.parquet')
df_endpoint_4.shape

(5589876, 4)

In [21]:
df_endpoint_4.sample(4)

,developer,recommend,release_year,sentiment_analysis
2596809,"Gearbox Software,Aspyr (Mac &amp; Linux)",True,2012,2
3016018,None,True,sin dato,2
2966402,Chucklefish,True,2016,2
4138509,tobyfox,True,2015,2


In [22]:
df_normalizado = df_endpoint_4[df_endpoint_4['release_year'] != 'sin dato']

In [23]:
df_normalizado.shape

(5270400, 4)

In [24]:
df_normalizado['release_year'] = df_normalizado['release_year'].astype(int)

C:\Users\pablo\AppData\Local\Temp\ipykernel_13384\2628637238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normalizado['release_year'] = df_normalizado['release_year'].astype(int)


In [26]:
df_normalizado.head(3)

,developer,recommend,release_year,sentiment_analysis
0,Valve,True,1998,2
1,Valve,True,1998,2
2,Valve,True,1998,2


In [27]:
df_normalizado.to_parquet('df_endpoint_4.parquet')